### Import Libraries

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.tools import tool
from dotenv import load_dotenv
import json

# Cargar API key desde .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

### Defines the functions to control the mixer

In [37]:
@tool
def adjust_volume(input: str) -> str:
    """
    Ajusta el volumen del canal.
    Formato esperado: '2 bajo' (canal nivel)
    """
    try:
        channel, level = input.strip().split()
        return f"✅ Volumen del canal {channel} ajustado a {level}"
    except Exception:
        return "❌ Formato incorrecto. Usa: '<canal> <nivel>'"

@tool
def adjust_eq(input: str) -> str:
    """
    Ajusta la ecualización.
    Formato: '2 graves subir'
    """
    try:
        channel, freq, action = input.strip().split()
        return f"✅ EQ canal {channel}: {freq} {action}"
    except Exception:
        return "❌ Formato inválido. Usa: '<canal> <frecuencia> <acción>'"

@tool
def mute_channel(input: str) -> str:
    """Silencia el canal especificado (ej: '3')"""
    return f"✅ Canal {input.strip()} silenciado"


tools = [adjust_volume, adjust_eq, mute_channel]

### Call llm

In [38]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=api_key,
    temperature=0.4
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={
        "prefix": "Sos un asistente de sonido que controla un mixer digital con comandos de texto. Podés ajustar volumen, EQ (graves, medios, agudos) y silenciar canales. Usá las herramientas disponibles para ejecutar acciones precisas, lo que retornes retornalo en formato json en LENGUAJE INGLES, en el cual retorne cosas como channel, eq, etc.",
    }
)


### This is the main function that calls the function to connect langchain to llm.

In [39]:
if __name__ == "__main__":
    print("🎛️ Mixer inteligente (Langchain + Gemini)")
    while True:
        user_input = input("📢 Instrucción: ")
        if user_input.lower() in ["exit", "salir"]:
            break
        result = agent.run(user_input)
        print("🤖 Acción tomada:", result)

🎛️ Mixer inteligente (Langchain + Gemini)
